In [2]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

# HIST_PRICE_DF = pd.read_excel (r'/Users/joezhou/Downloads/ALL_Prices.xlsx')
HIST_PRICE_DF_RAW = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_Prices.csv', sep='|')

#Create a sample for code testing purposes
# HIST_PRICE_DF = HIST_PRICE_DF_RAW[HIST_PRICE_DF_RAW['TickName'].isin(['BHP.AX','RIO.AX','TCL.AX','CBA.AX','MQG.AX','CSL.AX','NAB.AX','WBC.AX','SCG.AX','ANZ.AX','FMG.AX','WES.AX','TLS.AX','RMD.AX','WOW.AX','APA.AX','ATM.AX','GMG.AX','STO.AX','AMC.AX','MGR.AX','COL.AX','ALL.AX','NCM.AX','ZIP.AX'])]

HIST_PRICE_DF = HIST_PRICE_DF_RAW

#create a baselist for adding on the feature engineered variables
TICKER_LIST = HIST_PRICE_DF['TickName'].drop_duplicates().reset_index(drop=True)

In [3]:
# Create tag variables
# Calculated higher or lower
# daily change 
# daily change absolute


import numpy as np
HIST_PRICE_DF['Freq'] = 1


HIST_PRICE_DF['DAY_UP'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] > 0, 1, 0)
HIST_PRICE_DF['DAY_DWN'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] < 0, 1, 0)
HIST_PRICE_DF['DAY_FLAT'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] == 0, 1, 0)

HIST_PRICE_DF['DAY_CHANGE_RATE'] = HIST_PRICE_DF['Close'] / HIST_PRICE_DF['Open'] - 1
HIST_PRICE_DF['DAY_CHANGE_NUM'] = HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] 

HIST_PRICE_DF_SUM = HIST_PRICE_DF.groupby("TickName")["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT", "DAY_CHANGE_NUM", "DAY_CHANGE_RATE"].sum()

# calculate the magnitude of movement



# HIST_PRICE_DF['Mvmt_val'] = HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open']
# HIST_PRICE_DF['Mvmt_rate'] = HIST_PRICE_DF['Close'] / HIST_PRICE_DF['Open']-1

# tag events

# HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] > 0, 'DAY_UP_VAL'] = HIST_PRICE_DF['Mvmt_val']
# HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] > 0, 'DAY_UP_RATE'] = HIST_PRICE_DF['Mvmt_rate']

# down movements
# HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] < 0, 'DAY_DWN_VAL'] = HIST_PRICE_DF['Mvmt_val']
# HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] < 0, 'DAY_DWN_RATE'] = HIST_PRICE_DF['Mvmt_rate']

# Aggregate to ticker level

# HIST_PRICE_DF_SUM = HIST_PRICE_DF.groupby("TickName")["DAY_UP", "Freq", "DAY_UP_VAL", "DAY_UP_RATE" , "DAY_DWN_VAL","DAY_DWN_RATE" ].sum()
# HIST_PRICE_DF_MED = HIST_PRICE_DF.groupby("TickName")["DAY_UP_VAL", "DAY_UP_RATE", "DAY_DWN_VAL","DAY_DWN_RATE" ].median()
# HIST_PRICE_DF_AVG = HIST_PRICE_DF.groupby("TickName")["DAY_UP_VAL", "DAY_UP_RATE", "DAY_DWN_VAL","DAY_DWN_RATE" ].mean()


# def f(grp):
#     return pd.DataFrame({
#                 'N_DAYS':len(grp),
#                 'N_UP_DAYS': grp['DAY_UP'].sum(),
#                 'Percentage of Up days': grp['DAY_UP'].mean(),
#                 'Average daily change':grp['DAY_CHANGE_RATE'].mean(),
#                 'Median daily change':grp['DAY_CHANGE_RATE'].median(),
#                 }
                # ,

                # 'NEW_VAR': grp['C'].mean() / grp['X'].sum()},  # <--- here we access different columns...
                # index=[grp.name]
                # )
# ## -- End pasted text --

# HIST_PRICE_DF_SUM = HIST_PRICE_DF.groupby('TickName', as_index=False).apply(f)


HIST_PRICE_DF_FIRST = HIST_PRICE_DF.groupby("TickName").first()
HIST_PRICE_DF_LAST = HIST_PRICE_DF.groupby("TickName").last()

<ipython-input-3-b4abf3bd20ec>:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  HIST_PRICE_DF_SUM = HIST_PRICE_DF.groupby("TickName")["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT", "DAY_CHANGE_NUM", "DAY_CHANGE_RATE"].sum()


In [4]:
# create ticker level file

DF_PRICE_INFO = HIST_PRICE_DF_SUM[["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT"]].merge(HIST_PRICE_DF_FIRST[['Open']], on = 'TickName',how = 'left')
DF_PRICE_INFO = DF_PRICE_INFO.merge(HIST_PRICE_DF_LAST[['Close']], on = 'TickName',how = 'left')

# calculate at ticker level
DF_PRICE_INFO['PROB_HIGH'] = DF_PRICE_INFO['DAY_UP'] / DF_PRICE_INFO['Freq']
DF_PRICE_INFO['PROB_DWN'] = DF_PRICE_INFO['DAY_DWN'] / DF_PRICE_INFO['Freq']
DF_PRICE_INFO['PROB_FLAT'] = DF_PRICE_INFO['DAY_FLAT'] / DF_PRICE_INFO['Freq']

DF_PRICE_INFO['CHG_RTE_PRICE'] = DF_PRICE_INFO['Close'] / DF_PRICE_INFO['Open'] -1

del HIST_PRICE_DF_SUM, HIST_PRICE_DF_FIRST, HIST_PRICE_DF_LAST

In [5]:
DF_YFIN_INFO1 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_1.csv',sep='|') 
DF_YFIN_INFO2 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_2.csv',sep='|') 
DF_YFIN_INFO3 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_3.csv',sep='|') 
DF_YFIN_INFO4 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_4.csv',sep='|') 

DF_YFIN_INFO = pd.concat([DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3, DF_YFIN_INFO4], ignore_index=True)
# DF_YFIN_INFO = pd.concat([DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3], ignore_index=True)


del DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3, DF_YFIN_INFO4



In [6]:
# Only keep columns needed, selected after manual review

DF_YFIN_INFO_SUBSET = DF_YFIN_INFO[[
# Price / volume Movement	
'52WeekChange','fiftyDayAverage','fiftyTwoWeekHigh','fiftyTwoWeekLow','averageDailyVolume10Day','averageVolume10days',
#Location	
'address1','address2','city','state','country','zip','exchange',
# Dividend	
'dividendRate','dividendYield','exDividendDate','payoutRatio','trailingAnnualDividendRate','trailingAnnualDividendYield',
# Timing	
'mostRecentQuarter','lastFiscalYearEnd','lastSplitDate','nextFiscalYearEnd',
# Fundamentals	
'open','marketCap','sharesOutstanding','floatShares','bookValue','regularMarketPrice','regularMarketVolume','heldPercentInsiders','heldPercentInstitutions',
# Performance	
'earningsQuarterlyGrowth','netIncomeToCommon','beta','enterpriseToEbitda','enterpriseToRevenue','enterpriseValue','priceToBook','priceToSalesTrailing12Months','profitMargins','trailingEps','trailingPE','fullTimeEmployees',
# Future dated	
'forwardEps','forwardPE',
# Company Identifier	
'symbol','shortName','longName',
# ,'longBusinessSummary','website','messageBoardId',
# Index Grouping	
'industry','sector'
]]

DF_YFIN_INFO_SUBSET.rename(columns={"symbol": "TickName"}, inplace = True)


#Create a sample for code testing purposes
# DF_YFIN_INFO_SUBSET = DF_YFIN_INFO_SUBSET[DF_YFIN_INFO_SUBSET['TickName'].isin(['BHP.AX','RIO.AX','TCL.AX','CBA.AX','MQG.AX','CSL.AX','NAB.AX','WBC.AX','SCG.AX','ANZ.AX','FMG.AX','WES.AX','TLS.AX','RMD.AX','WOW.AX','APA.AX','ATM.AX','GMG.AX','STO.AX','AMC.AX','MGR.AX','COL.AX','ALL.AX','NCM.AX','ZIP.AX'])]


# DF_YFIN_INFO_SUBSET = DF_YFIN_INFO_SUBSET[['TickName', 'sector']]

In [7]:
# Add info onto prices data


DF_YFIN_INFO_SUBSET.set_index('TickName')
# DF_PRICE_INFO.set_index('TickName')


ADDED_PRICE_INF = DF_PRICE_INFO.merge(DF_YFIN_INFO_SUBSET, on = 'TickName',how = 'left')


In [11]:
# Build sector related models
MOD_DATA = ADDED_PRICE_INF[['TickName','PROB_HIGH','CHG_RTE_PRICE','sector']]

# padding rows into columns of 1 or 0
MOD_DATA = pd.concat([MOD_DATA, pd.get_dummies(MOD_DATA['sector'])], axis=1);

